In [26]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
from selenium.webdriver.common.action_chains import ActionChains


In [27]:
driver = webdriver.Chrome(ChromeDriverManager().install())
url1="https://www.getdoc.com/en/appointments/search/results?clinic_name=&proximity=10&specialisation_id=&country_code=MY&gender=&insurance_id=&date=&panel_id=&sort=NAME&mylocation=&search-type=type-clinic"
driver.get(url1)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Dhruv Bajaj\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [28]:
SCROLL_PAUSE_TIME = 0.2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [29]:
main_list=driver.find_elements_by_class_name("item")
print(len(main_list))

2031


In [30]:
doctors=main_list[1].find_element_by_id("list_doctor")
print(main_list[1].find_element_by_class_name("name").text)

[Qualitas Health] Ambulatory Care Centre @ ACC Evesuite Medical Centre


In [31]:
name_list=doctors.find_elements_by_class_name("row")
print( len(name_list))

10


In [32]:
for i in name_list:
    print(i.find_element_by_tag_name("a").text)
    print(i.find_element_by_tag_name("p").text)

Datin Dr Omkara Rubini Lingham
Ear, Nose & Throat (ENT)
Dr Asokumaran A/L Thanaraj
Eye - Ophthalmologist
Dr Chua Wei Siong
Orthopaedic Surgeon - Bone
Dr Gordon Pang Hwa
Geriatrician - Elderly Health, Physician (Internal Medicine)
Dr Muhammad Daniel Soma Kumar Bin Abdullah
General Surgeon
Dr Muhammad Niraj Bin Muhammad Feizal
Obstetrician & Gynaecologist
Dr Sandev Singh
Gastroenterologist - Gut, Physician (Internal Medicine)
Dr Shereen Shoba A/P Devaraj Nair
Ear, Nose & Throat (ENT)
Dr Vijeyasingam Rajasingam
General Surgeon, Vascular Surgeon
Dr Vishalkumar Dalichand
Obstetrician & Gynaecologist


In [33]:
print(main_list[0].find_element_by_class_name("address").text)

No. 4A, Jalan Lembah 19, Bandar Seri Alam, 81750 Masai, Johor


In [34]:
tel_list=main_list[0].find_elements_by_class_name("enquire-clinic-wrap")
tel=tel_list[1].find_element_by_tag_name("button").text


In [35]:
tel1=driver.find_element_by_id("modal-clinic-number")
#print(tel1.find_element_by_tag_name("a").text)


In [36]:

ActionChains(driver).move_by_offset(200, 100).click().perform()

In [37]:
main_list=driver.find_elements_by_class_name("item")
df_main=pd.DataFrame()
for i in range(len(main_list)):
    
    doctors=main_list[i].find_element_by_id("list_doctor")
    dr_name_list=doctors.find_elements_by_class_name("row")
    while True:
        address=main_list[i].find_element_by_class_name("address").text
        if(address!=''):
            break
    while True:
        clinic=main_list[i].find_element_by_class_name("name").text
        if(clinic!=''):
            break
    dr=[]
    dr_spe=[]
    dr_url=[]
    for j in dr_name_list:
        while True:
            drr=j.find_element_by_tag_name("a").text
            dr_spee=j.find_element_by_tag_name("p").text
            if(drr!='' and 'dr_spee'!=''):
                break
        url=j.find_element_by_tag_name("img")
        dr_url.append(url.get_attribute("src"))
        dr.append(drr)
        dr_spe.append(dr_spee)
    flag=False
    tel_list=main_list[i].find_elements_by_class_name("enquire-clinic-wrap")
    if(len(tel_list)>=2):
        b=tel_list[1].find_element_by_tag_name("button")
        if(b.text=="CALL"):
            b.click()
        else:
            continue
    elif(len(tel_list)==1):
        b=tel_list[0].find_element_by_tag_name("button")
        if(b.text=="CALL"):
            b.click()
        else:
            continue
    elif(len(tel_list)==0):
        flag=True
    
    
    driver.implicitly_wait(5)
    tel1=driver.find_element_by_id("modal-clinic-number")
    #driver.implicitly_wait(5)
    while True:
        tell=tel1.find_element_by_tag_name("a").text
        if(tell!='' or flag==True):
            break;
    
    element=driver.find_element_by_xpath("//button[text()='Close']")
    size = element.size
    w, h = size['width'], size['height']
    try:
        ActionChains(driver).move_by_offset(w,h).click().perform()
    except:
        ActionChains(driver).move_by_offset(300, 100).click().perform()
    
    Data={
        'clinic':clinic,
        'doctor-name':dr,
        'Speciality':dr_spe,
        'doctor_image':dr_url,
        'address':address,
        'telephone': tell
    }
    df_main=df_main.append(Data,ignore_index=True)

    

In [38]:
df_main

,Speciality,address,clinic,doctor-name,doctor_image,telephone
0,[Ambulance],"No. 4A, Jalan Lembah 19, Bandar Seri Alam, 817...",Lifesaver Ambulance Johor,[Dr Lifesaver Ambulance Johor],[https://www.getdoc.com/en/common/image/displa...,+6019 999 5800
1,"[Ear, Nose & Throat (ENT), Eye - Ophthalmologi...","Evesuite@Ara Damansara, A-1-01, Jln PJU 1/41A ...",[Qualitas Health] Ambulatory Care Centre @ ACC...,"[Datin Dr Omkara Rubini Lingham, Dr Asokumaran...",[https://www.getdoc.com/en/common/image/displa...,+603 7886 4916
2,[GP/Family Doctor],"No. A-5, Jalan Stadium, 25000 Kuantan",[Qualitas Health] Ananda Klinik,[Dr. Raj Kumar David],[https://www.getdoc.com/en/common/image/displa...,+09-5138 175
3,[GP/Family Doctor],"Bangsar Lot G116, Ground Floor, Bangsar Shoppi...",[Qualitas Health] Clinic @ Bangsar,[SUMAM MARY JOHN],[https://www.getdoc.com/en/common/image/displa...,+03-2011 4336
4,[GP/Family Doctor],"70-01-64, Level One, D’Piazza Mall, Jalan Mahs...",[Qualitas Health] Klinik Aman (Bandar Bayan Baru),[Dr Ho Yuen Yu],[https://www.getdoc.com/en/common/image/displa...,+04-6452793
...,...,...,...,...,...,...
2017,[GP/Family Doctor],"No.q002, Gf, Block Q, Jalan Pju 10/2b, Damansa...",Wong & Liew Clinic (damansara Damai),[Dr. Wong Kevin],[https://www.getdoc.com/en/common/image/displa...,+03-61575726
2018,[GP/Family Doctor],"Ad 102, Batu Kawah Newtownship, Jln Batu Kawa,...",Wong Family Clinic,[Wong Mee Heang],[https://www.getdoc.com/en/common/image/displa...,+6082 457815
2019,[Dentist - General],"25-2, Jalan Pju 8/5a, Damansara Perdana, Petal...",Yong Klinik Pakar Periodental Dan Pergigian De...,[Yong Hui Ping],[https://www.getdoc.com/en/common/image/displa...,+60 377105828
2020,[Aesthetic Doctor],"D-06-Gf, Garden Shoppe One City, Jalan Usj 25/...",Ys Clinic,[Yang Sheng Por],[https://www.getdoc.com/en/common/image/displa...,+60 380116708


In [39]:
Cardiologist_15_get_doc=df_main.copy()

In [40]:
Cardiologist_15_get_doc.to_csv('Cardiologist_15_get_doc.csv')